In [3]:
#import library: bs4, urllib.request, numpy, string, stopwords
import bs4 as bs
import urllib.request
import numpy as np 
import string 
from nltk.corpus import stopwords 

# with help of library urllib.request we can open website and do some operations like read
data = urllib.request.urlopen('https://en.wikipedia.org/wiki/Artificial_intelligence')
article = data.read()
# with help of library bs4 we can take lxml format. lxml - the most feature-rich and easy-to-use library for processing XML and HTML in the Python language.
parsedArticle = bs.BeautifulSoup(article,'lxml')
# from tag "p" we take all information
paragraphs = parsedArticle.find_all('p')

articleText = ""
# we combine all text
for p in paragraphs:
    articleText += p.text

In [4]:
# we calculate probability of word
def probabilityOfWord(x): 
    e_x = np.exp(x - np.max(x)) 
    return e_x / e_x.sum() 
# main class where we can train and predict data   
class word2vec(object): 
    #set parameters
    def __init__(self): 
        self.N = 10
        self.X_train = [] 
        self.y_train = [] 
        self.window_size = 2
        self.alpha = 0.001
        self.words = [] 
        self.word_index = {} 
   # V=Number of unique words in our corpus of text
# W=Weights between input layer and hidden layer
# W1=Weights between hidden layer and output layer
# N=Number of neurons in the hidden layer of neural network
    def initialize(self,V,data): 
        self.V = V 
        self.W = np.random.uniform(-0.8, 0.8, (self.V, self.N)) 
        self.W1 = np.random.uniform(-0.8, 0.8, (self.N, self.V)) 
           
        self.words = data 
        for i in range(len(data)): 
            self.word_index[data[i]] = i 
   
    # we calculate Forward Propagation   
    def forwardPropagation(self,X): 
        self.h = np.dot(self.W.T,X).reshape(self.N,1) 
        self.u = np.dot(self.W1.T,self.h) 
        self.y = probabilityOfWord(self.u)   
        return self.y 
    # we calculate Back Propagation       
    def backPropagation(self,x,t): 
        e = self.y - np.asarray(t).reshape(self.V,1) 
        # e.shape is V x 1 
        dLdW1 = np.dot(self.h,e.T) 
        X = np.array(x).reshape(self.V,1) 
        dLdW = np.dot(X, np.dot(self.W1,e).T) 
        self.W1 = self.W1 - self.alpha*dLdW1 
        self.W = self.W - self.alpha*dLdW 
    # in this function we train our data       
    def train(self,epochs): 
        for x in range(1,epochs):         
            self.loss = 0
            for j in range(len(self.X_train)): 
                self.forwardPropagation(self.X_train[j]) 
                self.backPropagation(self.X_train[j],self.y_train[j]) 
                C = 0
                for m in range(self.V): 
                    if(self.y_train[j][m]): 
                        self.loss += -1*self.u[m][0] 
                        C += 1
                self.loss += C*np.log(np.sum(np.exp(self.u))) 
            print("epoch ",x, " loss = ",self.loss) 
            self.alpha *= 1/( (1+self.alpha*x) ) 
     # in this function we predict words which similar and words often used together         
    def predict(self,word,numberOfPredictions): 
        if word in self.words: 
            index = self.word_index[word] 
            X = [0 for i in range(self.V)] 
            X[index] = 1
            prediction = self.forwardPropagation(X) 
            output = {} 
            for i in range(self.V): 
                output[prediction[i][0]] = i 
               
            topContextWords = [] 
            for k in sorted(output,reverse=True): 
                topContextWords.append(self.words[output[k]]) 
                if(len(topContextWords)>=numberOfPredictions): 
                    break
       
            return topContextWords 
        else: 
            print("Word not found in dicitonary") 
# we clean our data, we delete stopwords, space, punctuation, do lower words            
def preprocessing(corpus): 
    stopWords = set(stopwords.words('english'))     
    trainingData = [] 
    sentences = corpus.split(".") 
    for i in range(len(sentences)): 
        sentences[i] = sentences[i].strip() 
        sentence = sentences[i].split() 
        x = [word.strip(string.punctuation) for word in sentence 
                                     if word not in stopWords] 
        x = [word.lower() for word in x] 
        trainingData.append(x) 
    return trainingData 
       
# this function help us to take our data and train it, we divide words from sentences, sort, count  
def dataForTraining(sentences,w2v): 
    data = {} 
    for sentence in sentences: 
        for word in sentence: 
            if word not in data: 
                data[word] = 1
            else: 
                data[word] += 1
    V = len(data) 
    data = sorted(list(data.keys())) 
    vocab = {} 
    for i in range(len(data)): 
        vocab[data[i]] = i 
       
    for sentence in sentences: 
        for i in range(len(sentence)): 
            centerWord = [0 for x in range(V)] 
            centerWord[vocab[sentence[i]]] = 1
            context = [0 for x in range(V)] 
              
            for j in range(i-w2v.window_size,i+w2v.window_size): 
                if i!=j and j>=0 and j<len(sentence): 
                    context[vocab[sentence[j]]] += 1
            w2v.X_train.append(centerWord) 
            w2v.y_train.append(context) 
    w2v.initialize(V,data) 
   
    return w2v.X_train,w2v.y_train 

# we give a text from website
corpus = articleText
# quantity of how many we should train our data
epochs = 100
# our training data  
trainingData = preprocessing(corpus) 
# our method
w2v = word2vec() 
# our data and method  
dataForTraining(trainingData,w2v) 
#quantity of how many we should train our data
w2v.train(epochs)  
#find 3 close words for our "artificial" word  
print(w2v.predict("artificial",3))  

epoch  1  loss =  135279.29030424135
epoch  2  loss =  135163.84233265524
epoch  3  loss =  135050.21514731483
epoch  4  loss =  134938.24518554928
epoch  5  loss =  134827.77701046245
epoch  6  loss =  134718.66295077268
epoch  7  loss =  134610.76283738608
epoch  8  loss =  134503.9438806158
epoch  9  loss =  134398.0807434803
epoch  10  loss =  134293.05588510705
epoch  11  loss =  134188.7602779156
epoch  12  loss =  134085.09464885236
epoch  13  loss =  133981.97146769465
epoch  14  loss =  133879.31801735694
epoch  15  loss =  133777.08105100755
epoch  16  loss =  133675.23379133933
epoch  17  loss =  133573.78637985268
epoch  18  loss =  133472.80133367216
epoch  19  loss =  133372.41601955707
epoch  20  loss =  133272.87426843357
epoch  21  loss =  133174.56812871344
epoch  22  loss =  133078.08646898338
epoch  23  loss =  132984.25661605905
epoch  24  loss =  132894.1463288553
epoch  25  loss =  132808.9728760216
epoch  26  loss =  132729.87170863347
epoch  27  loss =  132657.